In [94]:
#import the necessary libraries
import pandas as pd   
import matplotlib.pyplot as plt
import numpy as np


from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics


In [95]:
df = pd.read_csv('train.csv')#reading the csv file train.csv
df_val = pd.read_csv('test.csv')#for validating our trained model by reading the test.csv


In [96]:
df.info()#this will give me the information of our table the datatypes the non null count as well

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 58 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   RecordID                       50000 non-null  int64  
 1   hospital_id                    50000 non-null  int64  
 2   icu_id                         50000 non-null  int64  
 3   ethnicity                      49424 non-null  object 
 4   gender                         49996 non-null  object 
 5   icu_admit_source               49953 non-null  object 
 6   icu_stay_type                  50000 non-null  object 
 7   icu_type                       50000 non-null  object 
 8   apache_3j_bodysystem           49724 non-null  object 
 9   apache_2_bodysystem            49724 non-null  object 
 10  age                            48199 non-null  float64
 11  elective_surgery               50000 non-null  int64  
 12  pre_icu_los_days               50000 non-null 

In [97]:
df.head()#prints the first 5 entries by default will give 5

,RecordID,hospital_id,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,...,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis,hospital_death
0,1,126,1931,Caucasian,M,Floor,transfer,Med-Surg ICU,Metabolic,Metabolic,...,86,93.0,86.0,NaN,NaN,0.01,0.00,0,0,0
1,2,112,1544,African American,M,Accident & Emergency,admit,Med-Surg ICU,Cardiovascular,Cardiovascular,...,95,95.0,95.0,333.0,3.7,NaN,NaN,0,0,0
2,3,153,1517,Caucasian,M,Floor,admit,MICU,Respiratory,Respiratory,...,162,174.0,162.0,160.0,4.2,0.38,0.15,0,0,0
3,4,109,1811,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Neurological,Neurologic,...,140,163.0,140.0,258.0,3.8,0.12,0.06,0,0,0
4,5,287,1845,Caucasian,F,Accident & Emergency,admit,CSICU,Cardiovascular,Cardiovascular,...,119,119.0,119.0,110.0,3.9,0.15,0.07,0,0,0


In [98]:
df.sample()#this will give any random numbered samples by default will give 1 

,RecordID,hospital_id,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,...,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis,hospital_death
1751,1752,248,1714,Caucasian,F,Floor,admit,MICU,Sepsis,Cardiovascular,...,82,82.0,82.0,113.0,3.9,0.15,0.05,0,0,0


In [99]:
df.isnull().sum()#this will give us the total null values in each column

RecordID                            0
hospital_id                         0
icu_id                              0
ethnicity                         576
gender                              4
icu_admit_source                   47
icu_stay_type                       0
icu_type                            0
apache_3j_bodysystem              276
apache_2_bodysystem               276
age                              1801
elective_surgery                    0
pre_icu_los_days                    0
apache_2_diagnosis                276
apache_3j_diagnosis                64
apache_post_operative               0
gcs_eyes_apache                   405
gcs_motor_apache                  405
gcs_unable_apache                  42
gcs_verbal_apache                 405
heart_rate_apache                   2
intubated_apache                    0
resprate_apache                    62
temp_apache                      1249
ventilated_apache                   0
d1_diasbp_min                       0
d1_diasbp_no

In [100]:
df_val.isnull().sum()#this will give us the total null values in each column

RecordID                            0
hospital_id                         0
icu_id                              0
ethnicity                         358
gender                              2
icu_admit_source                   26
icu_stay_type                       0
icu_type                            0
apache_3j_bodysystem              180
apache_2_bodysystem               180
age                              1111
elective_surgery                    0
pre_icu_los_days                    0
apache_2_diagnosis                180
apache_3j_diagnosis                47
apache_post_operative               0
gcs_eyes_apache                   228
gcs_motor_apache                  228
gcs_unable_apache                  19
gcs_verbal_apache                 228
heart_rate_apache                   1
intubated_apache                    0
resprate_apache                    46
temp_apache                       760
ventilated_apache                   0
d1_diasbp_min                       0
d1_diasbp_no

In [101]:
df.dtypes#the data types of each column

RecordID                           int64
hospital_id                        int64
icu_id                             int64
ethnicity                         object
gender                            object
icu_admit_source                  object
icu_stay_type                     object
icu_type                          object
apache_3j_bodysystem              object
apache_2_bodysystem               object
age                              float64
elective_surgery                   int64
pre_icu_los_days                 float64
apache_2_diagnosis               float64
apache_3j_diagnosis              float64
apache_post_operative              int64
gcs_eyes_apache                  float64
gcs_motor_apache                 float64
gcs_unable_apache                float64
gcs_verbal_apache                float64
heart_rate_apache                float64
intubated_apache                   int64
resprate_apache                  float64
temp_apache                      float64
ventilated_apach

In [102]:
df['age'].fillna(df['age'].median(), inplace=True)#filling in the missing values of age using median

In [103]:
#now to fill in the other numeric columns with the median
df['apache_2_diagnosis'].fillna(df['apache_2_diagnosis'].median(), inplace=True)

df['apache_3j_diagnosis'].fillna(df['apache_3j_diagnosis'].median(), inplace=True)

df['gcs_eyes_apache'].fillna(df['gcs_eyes_apache'].median(), inplace=True)

df['gcs_motor_apache'].fillna(df['gcs_motor_apache'].median(), inplace=True)

df['gcs_unable_apache'].fillna(df['gcs_unable_apache'].median(), inplace=True)

df['gcs_verbal_apache'].fillna(df['gcs_verbal_apache'].median(), inplace=True)

df['temp_apache'].fillna(df['temp_apache'].median(), inplace=True)

df['apache_4a_icu_death_prob'].fillna(df['apache_4a_icu_death_prob'].median(), inplace=True)

In [104]:
df['resprate_apache'].fillna(df['resprate_apache'].median(), inplace=True)

In [105]:
df['heart_rate_apache'].fillna(df['heart_rate_apache'].median(), inplace=True)

In [106]:
df['ethnicity'].fillna(df['ethnicity'].mode()[0], inplace=True)

df['icu_admit_source'].fillna(df['icu_admit_source'].mode()[0], inplace=True)

df['apache_3j_bodysystem'].fillna(df['apache_3j_bodysystem'].mode()[0], inplace=True)

df['apache_2_bodysystem'].fillna(df['apache_2_bodysystem'].mode()[0], inplace=True)

df['gender'].fillna(df['gender'].mode()[0], inplace=True)

In [107]:
df.isnull().sum()

RecordID                            0
hospital_id                         0
icu_id                              0
ethnicity                           0
gender                              0
icu_admit_source                    0
icu_stay_type                       0
icu_type                            0
apache_3j_bodysystem                0
apache_2_bodysystem                 0
age                                 0
elective_surgery                    0
pre_icu_los_days                    0
apache_2_diagnosis                  0
apache_3j_diagnosis                 0
apache_post_operative               0
gcs_eyes_apache                     0
gcs_motor_apache                    0
gcs_unable_apache                   0
gcs_verbal_apache                   0
heart_rate_apache                   0
intubated_apache                    0
resprate_apache                     0
temp_apache                         0
ventilated_apache                   0
d1_diasbp_min                       0
d1_diasbp_no

In [108]:
df.dropna(axis='columns')

,RecordID,hospital_id,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,...,d1_heartrate_max,d1_mbp_min,d1_sysbp_min,d1_sysbp_noninvasive_min,h1_sysbp_max,h1_sysbp_min,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis,hospital_death
0,1,126,1931,Caucasian,M,Floor,transfer,Med-Surg ICU,Metabolic,Metabolic,...,110,55,71,71.0,93,86,0.00,0,0,0
1,2,112,1544,African American,M,Accident & Emergency,admit,Med-Surg ICU,Cardiovascular,Cardiovascular,...,93,67,95,95.0,95,95,0.02,0,0,0
2,3,153,1517,Caucasian,M,Floor,admit,MICU,Respiratory,Respiratory,...,108,77,133,133.0,174,162,0.15,0,0,0
3,4,109,1811,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Neurological,Neurologic,...,77,104,140,140.0,163,140,0.06,0,0,0
4,5,287,1845,Caucasian,F,Accident & Emergency,admit,CSICU,Cardiovascular,Cardiovascular,...,163,63,90,90.0,119,119,0.07,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,110,1689,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Musculoskeletal/Skin,Undefined Diagnoses,...,89,56,84,84.0,98,98,0.01,0,0,0
49996,49997,155,1719,Caucasian,F,Floor,admit,Med-Surg ICU,Neurological,Neurologic,...,82,92,131,131.0,145,145,0.00,0,0,0
49997,49998,93,1912,Caucasian,F,Accident & Emergency,transfer,Med-Surg ICU,Sepsis,Cardiovascular,...,130,61,85,85.0,115,85,0.05,1,0,0
49998,49999,230,1720,Caucasian,M,Operating Room / Recovery,admit,Cardiac ICU,Cardiovascular,Cardiovascular,...,101,82,112,112.0,137,137,0.00,0,0,0


In [109]:
df_onehot = pd.get_dummies(df)
df_onehot.dtypes

RecordID                                     int64
hospital_id                                  int64
icu_id                                       int64
age                                        float64
elective_surgery                             int64
                                            ...   
apache_2_bodysystem_Renal/Genitourinary      uint8
apache_2_bodysystem_Respiratory              uint8
apache_2_bodysystem_Trauma                   uint8
apache_2_bodysystem_Undefined Diagnoses      uint8
apache_2_bodysystem_Undefined diagnoses      uint8
Length: 96, dtype: object

In [110]:
df_onehot.dropna(axis='columns')

,RecordID,hospital_id,icu_id,age,elective_surgery,pre_icu_los_days,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,gcs_eyes_apache,...,apache_2_bodysystem_Cardiovascular,apache_2_bodysystem_Gastrointestinal,apache_2_bodysystem_Haematologic,apache_2_bodysystem_Metabolic,apache_2_bodysystem_Neurologic,apache_2_bodysystem_Renal/Genitourinary,apache_2_bodysystem_Respiratory,apache_2_bodysystem_Trauma,apache_2_bodysystem_Undefined Diagnoses,apache_2_bodysystem_Undefined diagnoses
0,1,126,1931,28.0,0,0.009722,122.0,703.03,0,3.0,...,0,0,0,1,0,0,0,0,0,0
1,2,112,1544,69.0,0,0.038194,302.0,109.09,0,4.0,...,1,0,0,0,0,0,0,0,0,0
2,3,153,1517,84.0,0,11.171528,106.0,212.01,0,3.0,...,0,0,0,0,0,0,1,0,0,0
3,4,109,1811,59.0,0,0.090972,301.0,403.01,0,4.0,...,0,0,0,0,1,0,0,0,0,0
4,5,287,1845,85.0,0,0.031944,117.0,106.01,0,4.0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,110,1689,42.0,0,0.000000,308.0,1102.01,0,3.0,...,0,0,0,0,0,0,0,0,1,0
49996,49997,155,1719,61.0,0,4.824306,120.0,407.01,0,4.0,...,0,0,0,0,1,0,0,0,0,0
49997,49998,93,1912,50.0,0,0.000694,113.0,501.05,0,4.0,...,1,0,0,0,0,0,0,0,0,0
49998,49999,230,1720,54.0,1,0.420139,202.0,1211.01,1,4.0,...,1,0,0,0,0,0,0,0,0,0


In [111]:
X = df_onehot.loc[:, df_onehot.columns != 'hospital_death']
y = df_onehot[['hospital_death']]

In [112]:
print(y['hospital_death'].value_counts())

0    45662
1     4338
Name: hospital_death, dtype: int64


In [113]:
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.3, random_state=2)

In [114]:
dt_5_2 = DecisionTreeClassifier(max_depth=5)  
dt_5_2.fit(trainX,trainy)
md_probs = dt_5_2.predict_proba(testX)

ValueError: Input X contains NaN.
DecisionTreeClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
ffs = feature_selection.SequentialFeatureSelector(, k_features = 'best', forward=True)
ffs.fit(X, y)
features = list(ffs.k_feature_names_)
features = list(map(int, features))
df.fit